<span style="color:hotpink; font-size:40px; font-weight:bold;">Packages and imports</span>

In [36]:
# Configuration settings
!git config --global user.name "mauriekeppens"
!git config --global user.email "keppens_maurie@hotmail.com"
import gh_scoped_creds
%ghscopedcreds

<IPython.core.display.Javascript object>

The code 9727-7AC4 has been copied to your clipboard.
You have 15 minutes to go to https://github.com/login/device and paste it there.

Waiting......


Visit https://github.com/apps/leap-hub-push-access to manage list of repositories you can push to from this location
Tip: Use https:// URLs to clone and push to repos, not ssh URLs!


In [39]:
# Import packages
from xmip.preprocessing import combined_preprocessing
from xmip.postprocessing import concat_experiments, merge_variables
from xmip.utils import cmip6_dataset_id

from dask.diagnostics import ProgressBar
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
import pandas as pd
import intake

import gcsfs
fs = gcsfs.GCSFileSystem()

<span style="color:hotpink; font-size:40px; font-weight:bold;">Setting path for saving cleaned CMIP6 testbed files</span>

In [42]:
### set paths ###

# this is the path to the 1982-2023 testbed, already saved
# ensemble_dir = 'gs://leap-persistent/abbysh/pco2_all_members_1982-2023/post00_regridded_members'

# if you'd like to save your own version of the testbed, 
# for instance, change the year range (a few cells below) or include other variables,
# define your own path here:

your_username = "mauriekeppens" # leap pangeo bucket name, also should be your github username
ensemble_dir = f'gs://leap-persistent/{your_username}/Ensemble_Testbed_Analysis/00_regridded_members'

<span style="color:hotpink; font-size:40px; font-weight:bold;">Load CMIP6 datasets from cloud that satisfy our requirements</span>

<span style="color:lightblue; font-size:30px; font-weight:bold;">CMIP6 Earth System Models (ESMs) we want, called "source_id"</span>

In [43]:
# Define all ESMs that we want to use to make our reconstructions
source_id=['ACCESS-ESM1-5','UKESM1-0-LL','CMCC-ESM2','CESM2-WACCM','CESM2','CanESM5-CanOE','CanESM5','MPI-ESM1-2-LR','GFDL-ESM4']

<span style="color:lightblue; font-size:30px; font-weight:bold;">Searching CMIP6 catalog</span>

<span style="color:thistle; font-size:25px; font-weight:bold;">Definitions and units</span>

Note: To convert pascals to microatm: 1 atmosphere (atm) = 101325 pascal (Pa) and then 10^6uatm = 1atm.

| Our variable name | CMIP6 output name  | Description                      | Units                                  |
|-------------------|--------------------|----------------------------------|----------------------------------------|
| pCO2              | spco2              | sea surface co2 partial pressure |pascals, CONVERTS TO MICROATM LATER     |
| SST               | tos                | sea surface temperature          |degrees Celsius                         |
| SSS               | sos                | sea surface salinity             |.001 (parts per thousand)               |
| Chl               | chl                | sea surface chlorophyll          |kilograms per cubic meter               |
| MLD               | mlotst             | mixed layer depth                |meters (defined by sigma T criterion)   |

In [57]:
# filter the full catalog for data we could use

# This is the store for CMIP6 datasets that pass ingestion tests. For more information: https://github.com/leap-stc/cmip6-leap-feedstock
url = "https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog.json"
col = intake.open_esm_datastore(url)

##search for data##
cat = col.search(
    source_id = source_id, # ESM list
    variable_id=['tos', 'sos', 'chl', 'mlotst', 'spco2'], #variables we want, descriptions written above
    table_id=['Omon'], # monthly ocean output only
    experiment_id=['ssp245','historical'], # ssp scenario of choice, plus historical for pre-2014 model output
    # "each ensemble member has undergone the same forcing, namely solar, volcanic, and historical atmospheric CO2 through 2015, followed by RCP4.5 
    # through 2022 (according to the article, here 2023 also included)". 
    require_all_on=['source_id', 'member_id', 'grid_label'] # this ensures that results will have all variables and experiments available
)

print(cat.df)

      activity_id institution_id      source_id experiment_id  member_id  \
0     ScenarioMIP          CSIRO  ACCESS-ESM1-5        ssp245  r10i1p1f1   
1     ScenarioMIP          CSIRO  ACCESS-ESM1-5        ssp245  r10i1p1f1   
2     ScenarioMIP          CSIRO  ACCESS-ESM1-5        ssp245  r10i1p1f1   
3            CMIP          CSIRO  ACCESS-ESM1-5    historical  r10i1p1f1   
4     ScenarioMIP          CSIRO  ACCESS-ESM1-5        ssp245  r10i1p1f1   
...           ...            ...            ...           ...        ...   
1206         CMIP           MOHC    UKESM1-0-LL    historical   r8i1p1f2   
1207         CMIP           MOHC    UKESM1-0-LL    historical   r8i1p1f2   
1208  ScenarioMIP           MOHC    UKESM1-0-LL        ssp245   r8i1p1f2   
1209  ScenarioMIP           MOHC    UKESM1-0-LL        ssp245   r8i1p1f2   
1210         CMIP           MOHC    UKESM1-0-LL    historical   r8i1p1f2   

     table_id variable_id grid_label sub_experiment_id variant_label  \
0        Omon  

<span style="color:thistle; font-size:25px; font-weight:bold;">To view available datasets, number of members per ESM</span>

In [61]:
members_count = cat.df.groupby(['source_id', 'grid_label','table_id'])[['member_id']].nunique()
print(members_count)
# grid_label = gn -> model's original grid

# Calculate total number of members overall 
total_members = members_count['member_id'].sum()
print("Total ensemble members:", total_members)

                                   member_id
source_id     grid_label table_id           
ACCESS-ESM1-5 gn         Omon             35
CESM2         gn         Omon              3
CESM2-WACCM   gn         Omon              3
              gr         Omon              3
CMCC-ESM2     gn         Omon              1
CanESM5       gn         Omon             20
CanESM5-CanOE gn         Omon              3
GFDL-ESM4     gr         Omon              1
MPI-ESM1-2-LR gn         Omon             47
UKESM1-0-LL   gn         Omon              5
Total ensemble members: 121


<span style="color:hotpink; font-size:40px; font-weight:bold;">Regridding</span>

<span style="color:lightblue; font-size:30px; font-weight:bold;">Turn catalog of ESM datasets into dictionary</span>

In [63]:
##turn data into dataset dictionary##
ddict = cat.to_dataset_dict(            # loads the selected datasets into a Python dictionary of xarray.Dataset objects
    preprocess=combined_preprocessing,  # combined_preprocessing is a Python function (or sometimes a small chain of functions) that is applied to each dataset
                                        # Its role is to clean, standardize, or subset the data before you start working with it (Renaiming variables to standard names, handling missing values...)
    xarray_open_kwargs=dict(use_cftime=True),
    aggregate=False                  # not to combine datasets from multiple files automatically.
)

## some arrays have an "area" variable. This is to drop that variable:
for item in ddict:
    if 'area' in ddict[item]:
        ddict[item] = ddict[item].drop_vars('area')

## ignore the warnings!



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.sub_experiment_id.variant_label.version.zstore'


/srv/conda/envs/notebook/lib/python3.12/site-packages/xmip/preprocessing.py:229: UserWarning: ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp245.r2i1p1f1.Omon.gn.none.chl: Unit correction failed with: Cannot convert variables:
    incompatible units for variable ('lev',): cannot use 'm' as a unit
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xmip/preprocessing.py:229: UserWarning: ScenarioMIP.NCAR.CESM2.ssp245.r4i1p1f1.Omon.gn.none.sos: Unit correction failed with: Cannot parse units:
    invalid units for variable 'sos': 0.001 (attribute) (reason: Unit expression cannot have a scaling factor.)
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xmip/preprocessing.py:229: UserWarning: ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp245.r16i1p1f1.Omon.gn.none.sos: Unit correction failed with: Cannot parse units:
    invalid units for variable 'sos': 0.001 (attribute) (reason: Unit expression cannot have a scaling factor.)
  warnings.warn(
/srv/conda/envs/notebook/lib/

<span style="color:lightblue; font-size:30px; font-weight:bold;">Filltering out members with buggy times</span>

What the following hack does is take the "time" data from a historical member, and a scenario member, that have no bugs.

We then apply the non-buggy historical "time" data to all the historical members, and apply the non-buggy scenario "time" data to all the scenario members. 

If a member does not satisfy the necessary requirements (historical "time" data starts in 1850 and ends in 2014, for example), it is removed from our list of members.

In [68]:
## temporary time hack due to time bugs ###

hist_time = ddict['CMIP.CCCma.CanESM5.historical.r6i1p1f1.Omon.sos.gn.none.r6i1p1f1.v20190429.gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/historical/r6i1p1f1/Omon/sos/gn/v20190429/'].time
fut_time = ddict['ScenarioMIP.CCCma.CanESM5.ssp245.r3i1p1f1.Omon.sos.gn.none.r3i1p1f1.v20190429.gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp245/r3i1p1f1/Omon/sos/gn/v20190429/'].time
dict_list = list(ddict.values())
for item in dict_list:
    if item.time.data[0].year == 1850 and item.time.data[-1].year == 2014:
        item['time'] = hist_time
    elif item.time.data[0].year == 2015 and item.time.data[-1].year == 2100:
        item['time'] = fut_time

print(hist_time)
print(fut_time)

<xarray.DataArray 'time' (time: 1980)> Size: 16kB
array([cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 16, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeNoLeap(2014, 10, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 11, 16, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 12, 16, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    long_name:      time
    standard_name:  time
<xarray.DataArray 'time' (time: 1032)> Size: 8kB
array([cftime.DatetimeNoLeap(2015, 1, 16, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2015, 2, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2015, 3, 16, 12, 0, 0, 0, ha

In [87]:
## functions like xarray.concat: chains together historical + scenario for each member, so time span is 1850-2100
ds = concat_experiments(ddict)

len(ds) # The length of ds is number of members (121) times number of variables (5: chl, SST, SSS, MLD, pCO2)

# Look at the first key and see how it is structured
first_key = list(ds.keys())[0]
first_dataset = ds[first_key]
print(f"Dataset key: {first_key}")
print(first_dataset)        
print("Dims:", first_dataset.dims)
print("Data variables:", list(first_dataset.data_vars))
print("Coordinates:", list(first_dataset.coords))

# the first key summarizes the information for the MPI-ESM1-2-LR ESM, member r2i1p1f1, for chl-a
# also depth profiles (lev)
# Coordinates: ['lat', 'lev', 'lon', 'time', 'lat_verticies', 'lon_verticies', 'lev_bounds', 'time_bounds', 'y', 'x', 'lon_bounds', 'lat_bounds', 'variant_label', 'sub_experiment_id']

Dataset key: MPI-ESM1-2-LR.gn.Omon.r2i1p1f1.chl
<xarray.Dataset> Size: 27GB
Dimensions:            (variant_label: 1, sub_experiment_id: 1, time: 3012,
                        lev: 40, y: 220, x: 256, vertex: 4, bnds: 2)
Coordinates: (12/14)
    lat                (y, x) float64 451kB dask.array<chunksize=(220, 256), meta=np.ndarray>
  * lev                (lev) float64 320B 6.0 17.0 27.0 ... 5.17e+03 5.72e+03
    lon                (y, x) float64 451kB dask.array<chunksize=(220, 256), meta=np.ndarray>
  * time               (time) object 24kB 1850-01-16 12:00:00 ... 2100-12-16 ...
    lat_verticies      (y, x, vertex) float64 2MB dask.array<chunksize=(220, 256, 4), meta=np.ndarray>
    lon_verticies      (y, x, vertex) float64 2MB dask.array<chunksize=(220, 256, 4), meta=np.ndarray>
    ...                 ...
  * y                  (y) int64 2kB 0 1 2 3 4 5 6 ... 214 215 216 217 218 219
  * x                  (x) int64 2kB 0 1 2 3 4 5 6 ... 250 251 252 253 254 255
    lon_bounds     

In [89]:
## functions like xarray.merge: combines separate variables into one dataset per member

ds = merge_variables(ds) 

# lots of warnings right now, but should be fine! the warnings mean the bad members are getting removed
# TODO: I think we can fix this bug by doing a time slice for when we want on each dataset BEFORE merging

# Show the structure
first_key = list(ds.keys())[0]
first_dataset = ds[first_key]
print(f"Dataset key: {first_key}")
print(first_dataset)                
print("Dims:", first_dataset.dims)  
print("Data variables:", list(first_dataset.data_vars))  # should be: SST, SSS, MLD, chl-a and pCO2
print("Coordinates:", list(first_dataset.coords))      

Dataset key: MPI-ESM1-2-LR.gn.Omon.r2i1p1f1
<xarray.Dataset> Size: 30GB
Dimensions:            (variant_label: 1, sub_experiment_id: 1, time: 3012,
                        lev: 40, y: 220, x: 256, vertex: 4, bnds: 2)
Coordinates: (12/14)
    lat                (y, x) float64 451kB dask.array<chunksize=(220, 256), meta=np.ndarray>
  * lev                (lev) float64 320B 6.0 17.0 27.0 ... 5.17e+03 5.72e+03
    lon                (y, x) float64 451kB dask.array<chunksize=(220, 256), meta=np.ndarray>
  * time               (time) object 24kB 1850-01-16 12:00:00 ... 2100-12-16 ...
    lat_verticies      (y, x, vertex) float64 2MB dask.array<chunksize=(220, 256, 4), meta=np.ndarray>
    lon_verticies      (y, x, vertex) float64 2MB dask.array<chunksize=(220, 256, 4), meta=np.ndarray>
    ...                 ...
  * y                  (y) int64 2kB 0 1 2 3 4 5 6 ... 214 215 216 217 218 219
  * x                  (x) int64 2kB 0 1 2 3 4 5 6 ... 250 251 252 253 254 255
    lon_bounds         

<span style="color:lightblue; font-size:30px; font-weight:bold;">Setting up target grid</span>

In [91]:
## create desired resolution and time frame 
# This is set up for -180 thru 180 degrees for longitude, -90 thru 90 degrees for latitude 
# ^ As opposed to 0 thru 360 for longitude, for example

ylat = xr.DataArray(data=[x+.5 for x in range(-90, 90, 1)], dims=['ylat'], coords=dict( ylat=(['ylat'],[x+.5 for x in range(-90, 90, 1)]) ),)
xlon = xr.DataArray(data=[x+.5 for x in range(-180,180,1)], dims=['xlon'], coords=dict( xlon=(['xlon'],[x+.5 for x in range(-180,180,1)]) ),)
# alternatively: xlon = xr.DataArray(data=[x+.5 for x in range(0,360,1)], dims=['xlon'], coords=dict( xlon=(['xlon'],[x+.5 for x in range(0,360,1)]) ),)

## desired start and end 'year-month' for testbed 
processed_start_yearmonth = '1982-02'
processed_end_yearmonth = '2023-12'

## desired start and end year for testbed
# init_year = 1982
# fin_year = 2023

## time should be monthly on the middle of the month ('freq = "MS") refers to "month start" frequency, and we add 14 days to get to mid-month
# note that the time doesnt affect regridding but we do use this time to overwrite the monthly dates so its consistent
ttime = pd.date_range(start=str(processed_start_yearmonth), end=str(processed_end_yearmonth),freq='MS') + np.timedelta64(14, 'D') 

## set up our desired grid. It must be named this way for old XESFM versions
target_grid = xr.Dataset({'time':(['time'],ttime.values), 'latitude':(['latitude'],ylat.values),'longitude':(['longitude'],xlon.values)}) 

#print(ylat)
#print(xlon)
#print(ttime)
print(target_grid) # 3D with time x lat x lon

<xarray.Dataset> Size: 8kB
Dimensions:    (time: 503, latitude: 180, longitude: 360)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1982-02-15 1982-03-15 ... 2023-12-15
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
Data variables:
    *empty*


<span style="color:lightblue; font-size:30px; font-weight:bold;">Functions for regridding data</span>

In [92]:
def replace_calendar(ds:xr.Dataset) -> xr.Dataset:
    """
    Sets new time data for xarray dataset, according to target grid defined above.
    The purpose is to ensure the time coordinate has a consistent calendar.
    
    Args:
        ds (xr.Dataset): Initial dataset with times to be fixed.
    Returns: 
        ds (xr.Dataset): Dataset with fixed times.
    """
    year = ds.time.data[0].year
    month = ds.time.data[0].month
    start_date = f'{year}-{month:0>2}-01'
    new_monthly_time = xr.cftime_range(start_date, periods=len(ds.time), freq='1MS')
    ds = ds.assign_coords(time=new_monthly_time)
    return ds

#TODO:  create a regridder dict per source_id (faster)

# target_grid = xe.util.grid_global(1,1) # this is old, from julius
def regrid(target_grid, ds:xr.Dataset) -> xr.Dataset:
    """
    Regrids dataset to match times/lats/lons we want.

    Args:
        target_grid (xr.Dataset): Ideal dataset example in terms of format of times/lat/lon.
        ds (xr.Dataset): Dataset to regrid to match format of target_grid.
    Returns:
        ds_regridded (xr.Dataset): Dataset changed to match target_grid format.
    """
    #FIXME: This should not be done for every dataset
    regridder = xe.Regridder(ds, target_grid,         # regridding ds to the target_grid
                             'bilinear',              # interpolation method, other options include conservative
                             ignore_degenerate=True,  # skips any degenerate grid cells (You skip broken cells instead of erroring)
                             periodic=True)           # treats the longitude as circular (good for global datasets)
    #TODO: Check if this should be conservative?
    ds_regridded = regridder(ds, keep_attrs=True)     # preserves metadata/attributes from the original dataset.
    
    return ds_regridded

def full_testbed_processing(target_grid, init_year_month, fin_year_month, timespan, ds:xr.Dataset) -> xr.Dataset:
    """

    Args:
        target_grid (xr.Dataset): Ideal dataset example in terms of format of times/lat/lon.
        init_year_month (str): Initial year/month included in dataset.
        fin_year_month (str): Final year/month included in dataset.
        timespan (int): Number of months in dataset (time length of dataset).
        ds (xr.Dataset): Dataset to regrid to match format of target_grid.

    Returns:
        ds_new_cal (xr.Dataset): Dataset gridded to match target_grid format, with changed calendar and removed extraneous variables.
    """
    ds = ds.squeeze(drop=True)
    # If the dataset has depth levels (lev), keep only the surface (level 0) and drop the lev dimension.
    if 'lev' in ds.dims: 
        ds = ds.isel(lev=0).drop_vars(('lev'))
    # Select the correct time window
    ds = ds.sel(time=slice(init_year_month,fin_year_month))
    # testing
    assert len(ds.time) == timespan # Makes sure the dataset really has the expected number of time steps.
    assert ds.time.data[0].year == int(init_year_month[0:4]) # Makes sure the first year matches what you asked for
     
    # Processing
    ds_regridded = regrid(target_grid, ds) # function that regrids the original grid to the target grid
    ds_new_cal = replace_calendar(ds_regridded) # function to fix the calendar

    return ds_new_cal

<span style="color:lightblue; font-size:30px; font-weight:bold;">Regridding all members and saving them</span>

In [ ]:
member_counter = 0

## loop through all members (this is a combination of ESM and a member of that ESM)
for k,item in ds.items():
        print(f"Processing member no.{member_counter},{k}")

        ## regridding step here
        item_out = full_testbed_processing(target_grid, processed_start_yearmonth, processed_end_yearmonth, len(ttime), item) 
                # Runs your cleaning/regridding pipeline on each dataset.
                # applying the cleaning/regridding pipeline to the entire dataset for one ESM member, i.e. all its variables at once. (chla, SST, SSS, MLD, pCO2)
                # item_out.keys() -> chl, SSS, SST, MLD, pCO2
    
        ## get CMIP6 ID for member
        item_id = cmip6_dataset_id(item_out, id_attrs=[
            'source_id',
            'variant_label',
            'table_id'
        ])

        ## converting pco2 to microatmospheres, from pascals!
        if 'spco2' in list(item_out.keys()):
            if item_out['spco2'].attrs['units'] == 'Pa':
                print('fixing spco2 units')
                new_spco2 = item_out.spco2 * 10**6 / 101325  #to get to microatm
                item_out['spco2'].values = new_spco2
                item_out['spco2'].attrs["units"] = 'microatmospheres'
        
        ### removing unneccessary variables ###
        if 'lev_bounds' in item_out:
            item_out = item_out.drop_vars(('lev_bounds'))
        if 'time_bounds' in item_out:
            item_out = item_out.drop_vars(('time_bounds'))
        if 'lev_partial' in item_out:
            item_out = item_out.drop_vars(('lev_partial'))
        if 'nbnd' in item_out:
            item_out = item_out.drop_vars(('nbnd'))
        if 'lev_partial' in item_out.chl.dims:
            item_out['chl'] = item_out['chl'].sel({'lev_partial':1},drop=True)
    
        ## fixing variable/coord names ###
        fixed_names = ['ylat','xlon','mld','sss','sst']
        old_names = ['latitude','longitude','mlotst','sos','tos']
        
        for new,old in zip(fixed_names,old_names):
            item_out = item_out.rename({old:new})
        
        ## calculate pco2-T (temperature component of pco2) and pco2-residual (or "non-T pco2")
        pco2_T_calc = item_out['spco2'].mean('time') * np.exp(0.0423 * (item_out['sst'] - item_out['sst'].mean("time"))) 
                        # the part of pCO₂ variability that can be explained just by temperature effects explained by an exponential relationship
        pco2_resid_calc = item_out['spco2'] - pco2_T_calc # pCO2 residual -> our target variable
        
        item_out = item_out.assign(pco2_T = pco2_T_calc)               # add temporal component of pCO2 as variable 
        item_out = item_out.assign(pco2_residual = pco2_resid_calc)    # add the pCO2 residual as variable

        ## to make sure dimensions are lined up for all variables
        item_out = item_out.transpose('time','ylat','xlon')

        ## save regridded member data
        save_path = f"{ensemble_dir}/{item.attrs['source_id']}/member_{item.attrs['intake_esm_attrs:member_id']}/{item_id}.zarr"
                        # saved in: ESM model > Member ID name > ESM_member.Omon.zarr
        print(f"Writing to {save_path = }")
        with ProgressBar():
            item_out.chunk({'time':200}).to_zarr(save_path, mode='w')
        member_counter +=1

Processing member no.0,MPI-ESM1-2-LR.gn.Omon.r2i1p1f1


/tmp/ipykernel_3480/3573049386.py:14: DeprecationWarning: cftime_range() is deprecated, please use xarray.date_range(..., use_cftime=True) instead.
  new_monthly_time = xr.cftime_range(start_date, periods=len(ds.time), freq='1MS')


fixing spco2 units
Writing to save_path = 'gs://leap-persistent/mauriekeppens/Ensemble_Testbed_Analysis/00_regridded_members/MPI-ESM1-2-LR/member_r2i1p1f1/MPI-ESM1-2-LR.r2i1p1f1.Omon.zarr'
[                                        ] | 0% Completed | 287.35 us

/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/api/asynchronous.py:227: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


[########################################] | 100% Completed | 17.68 s
Processing member no.1,UKESM1-0-LL.gn.Omon.r8i1p1f2


...

[########################################] | 100% Completed | 16.21 s
Processing member no.104,MPI-ESM1-2-LR.gn.Omon.r35i1p1f1


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/computation/apply_ufunc.py:445: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 6.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (110, 128).
  result_vars[name] = func(*variable_args)
/tmp/ipykernel_3480/3573049386.py:14: DeprecationWarning: cftime_range() is deprecated, please use xarray.date_range(..., use_cftime=True) instead.
  new_monthly_time = xr.cftime_range(start_date, periods=len(ds.time), freq='1MS')


fixing spco2 units
Writing to save_path = 'gs://leap-persistent/mauriekeppens/Ensemble_Testbed_Analysis/00_regridded_members/MPI-ESM1-2-LR/member_r35i1p1f1/MPI-ESM1-2-LR.r35i1p1f1.Omon.zarr'


/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/api/asynchronous.py:227: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


[########################################] | 100% Completed | 16.45 s
Processing member no.105,MPI-ESM1-2-LR.gn.Omon.r47i1p1f1


In [ ]:
fs.ls('gs://leap-persistent/mauriekeppens/Ensemble_Testbed_Analysis/00_regridded_members')

In [7]:
all_members = []
for model_folder in fs.ls('gs://leap-persistent/mauriekeppens/Ensemble_Testbed_Analysis/00_regridded_members'):
    members = fs.ls(model_folder)
    all_members.extend(members)

print("Number of saved ESM members:", len(all_members))

['leap-persistent/mauriekeppens/pco2_residual/post00_regridded_members/CESM2/member_r10i1p1f1/CESM2.r10i1p1f1.Omon.zarr']

In [41]:
fs.ls('gs://leap-persistent/mauriekeppens')

['leap-persistent/mauriekeppens/pco2_residual']